In [ ]:
import tensorflow as tf, os

# 2.1 Load your trained model
model = tf.keras.models.load_model('best_sports_classifier.keras')

# 2.2 Export under versioned folder
export_dir = os.path.join('models', 'sports_classifier', '1')
os.makedirs(export_dir, exist_ok=True)

# Keras 3+: use .export() for SavedModel
model.export(export_dir)
print("✅ Exported SavedModel to", export_dir)


In [1]:
import os
os.makedirs('static', exist_ok=True)

html = """<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Sports Classifier</title>
  <style>
    /* Reset & base */
    * { box-sizing: border-box; margin: 0; padding: 0; }
    body {
      font-family: 'Segoe UI', Roboto, sans-serif;
      background: #f4f7fc;
      color: #333;
      line-height: 1.6;
      padding: 2em;
    }
    h1 {
      text-align: center;
      margin-bottom: 1em;
      color: #2c3e50;
    }

    /* File input + button */
    .controls {
      display: flex;
      justify-content: center;
      gap: 1em;
      margin-bottom: 1.5em;
    }
    input[type="file"] {
      padding: .5em;
      border: 1px solid #ccc;
      border-radius: 4px;
      background: white;
    }
    button {
      padding: .6em 1.2em;
      background: #3498db;
      border: none;
      border-radius: 4px;
      color: white;
      font-weight: 600;
      cursor: pointer;
      transition: background .2s;
    }
    button:disabled { background: #95a5a6; cursor: default; }
    button:hover:not(:disabled) { background: #2980b9; }

    /* Preview + results grid */
    .grid {
      display: grid;
      /* shrink min column width to 100px so more cards fit */
      grid-template-columns: repeat(auto-fit, minmax(100px, 1fr));
      gap: 0.8em;
    }
    .grid img {
      max-width: 100%;
      max-height: 80px;      /* limit height */
      object-fit: cover;     /* crop if needed */
      border-radius: 4px;
      border: 1px solid #ddd;
      display: block;
      margin: 0 auto;
    }

    /* Card for result */
    .result {
      background: white;
      border: 1px solid #e1e4e8;
      border-radius: 6px;
      padding: .5em;
      text-align: center;
      box-shadow: 0 1px 3px rgba(0,0,0,.1);
    }
    .result strong {
      display: block;
      margin-top: .5em;
      color: #2c3e50;
    }

    /* Status message */
    #status {
      text-align: center;
      margin-bottom: 1em;
      color: #555;
    }
  </style>
</head>
<body>
  <h1>Sports Image Classifier</h1>

  <div class="controls">
    <input id="files" type="file" accept="image/*" multiple>
    <button id="btn">Predict</button>
  </div>

  <div id="status"></div>

  <div id="preview" class="grid"></div>
  <div id="results" class="grid"></div>

  <script>
    const inp    = document.getElementById('files'),
          prev   = document.getElementById('preview'),
          res    = document.getElementById('results'),
          status = document.getElementById('status'),
          btn    = document.getElementById('btn');

    inp.onchange = () => {
      prev.innerHTML = '';
      res.innerHTML  = '';
      status.textContent = '';
      for (let f of inp.files) {
        let img = new Image();
        img.src = URL.createObjectURL(f);
        prev.appendChild(img);
      }
    };

    btn.onclick = async () => {
      if (!inp.files.length) {
        alert('Please select one or more images.');
        return;
      }
      btn.disabled = true;
      status.textContent = 'Uploading & predicting…';
      res.innerHTML = '';

      const fd = new FormData();
      for (let f of inp.files) fd.append('images', f);

      try {
        const r    = await fetch('/predict', { method:'POST', body:fd });
        const text = await r.text();
        let data;
        try { data = JSON.parse(text); } catch { data = null; }

        if (!r.ok) {
          status.textContent = 'Error during prediction';
          res.innerHTML = data
            ? `<pre style="color:red">${JSON.stringify(data, null, 2)}</pre>`
            : `<pre style="color:red">${text}</pre>`;
        } else {
          status.textContent = 'Prediction results:';
          data.predictions.forEach((lab, i) => {
            const card = document.createElement('div');
            card.className = 'result';
            card.innerHTML = `
              <img src="${URL.createObjectURL(inp.files[i])}">
              <strong>${lab}</strong>
            `;
            res.appendChild(card);
          });
        }
      } catch (e) {
        status.textContent = 'Network error';
        res.innerHTML = `<pre style="color:red">${e.message}</pre>`;
      } finally {
        btn.disabled = false;
      }
    };
  </script>
</body>
</html>
"""

with open('static/index.html', 'w', encoding='utf-8') as f:
    f.write(html)

print("✅ Written static/index.html with smaller, fitted images")


✅ Written static/index.html with smaller, fitted images


In [ ]:
from flask import Flask, request, jsonify
import tensorflow as tf
import numpy as np
import cv2
import traceback
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# ── 1) Load your trained model ───────────────────────────────────────────────
model = tf.keras.models.load_model('best_sports_classifier.keras')

# ── 2) Fit a LabelEncoder on your actual training labels ────────────────────
#    so inverse_transform gives exactly the classes you trained with.
train_df = pd.read_csv('dataset/train.csv')  # adjust path if needed
le = LabelEncoder().fit(train_df['label'])

# ── 3) Normalization stats (must match your train-time preprocessing) ───────
TRAIN_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
TRAIN_STD  = np.array([0.229, 0.224, 0.225], dtype=np.float32)

app = Flask(__name__, static_folder='static')

def preprocess_image(img_bytes):
    arr = np.frombuffer(img_bytes, dtype=np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError("Could not decode image (corrupt or wrong format)")
    img = cv2.resize(img, (64, 64))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255.0
    img = (img - TRAIN_MEAN) / TRAIN_STD
    return img.astype(np.float32)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        files = request.files.getlist('images')
        if not files:
            return jsonify({'error': 'No images provided'}), 400

        # Build batch
        batch = np.stack([preprocess_image(f.read()) for f in files], axis=0)

        # Model inference (no progress bar)
        preds = model.predict(batch, verbose=0)
        idxs  = preds.argmax(axis=1)

        # Map back to your actual labels
        labels = le.inverse_transform(idxs)

        return jsonify({'predictions': labels.tolist()})

    except Exception as e:
        traceback.print_exc()
        tb = traceback.format_exc().splitlines()[-5:]
        return jsonify({'error': str(e), 'traceback': tb}), 500

@app.route('/')
def home():
    return app.send_static_file('index.html')

if __name__ == '__main__':
    # disable reloader if you're running inside Jupyter / certain IDEs
    app.run(host='127.0.0.1', port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/May/2025 04:33:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2025 04:33:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2025 04:33:54] "GET / HTTP/1.1" 304 -
127.0.0.1 - - [05/May/2025 04:34:01] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2025 04:35:41] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2025 04:36:19] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2025 04:36:45] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2025 04:37:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2025 04:39:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2025 04:48:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2025 05:19:43] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2025 05:28:39] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2025 05:29:13] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2025 05:29:41] "POST /predict HTTP/1.1" 200 -
127.0.0